In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
df = sns.load_dataset('tips')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [ ]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
# use 4 boosting models on this dataset on regressors.
# RandomForest, Adaboost, Naive Bayes, QDA, Adaboost

In [ ]:
df.isna().sum()

,0
total_bill,0
tip,0
sex,0
smoker,0
day,0
time,0
size,0


In [ ]:
[df[i].value_counts() for i in "sex	smoker	day	time".split()]


[sex
 Male      157
 Female     87
 Name: count, dtype: int64,
 smoker
 No     151
 Yes     93
 Name: count, dtype: int64,
 day
 Sat     87
 Sun     76
 Thur    62
 Fri     19
 Name: count, dtype: int64,
 time
 Dinner    176
 Lunch      68
 Name: count, dtype: int64]

In [ ]:
df[['total_bill','size','tip']].corr()['tip']

,tip
total_bill,0.675734
size,0.489299
tip,1.000000


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
dfcopy = df.copy()

df['sex'] = le.fit_transform(dfcopy['sex'])
df['smoker'] = le.fit_transform(dfcopy['smoker'])
df['day'] = le.fit_transform(dfcopy['day'])
df['time'] = le.fit_transform(dfcopy['time'])

df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,0,0,2,0,2
1,10.34,1.66,1,0,2,0,3
2,21.01,3.50,1,0,2,0,3
3,23.68,3.31,1,0,2,0,2
4,24.59,3.61,0,0,2,0,4


In [ ]:
from scipy.stats import f_oneway

def checkanova(df, cat, target):
  p = f_oneway(*df.groupby(cat)[target].apply(list))[1]


  if p >0.05 : print(f"p = {p} ,  accept H0, no correlation between {cat} and {target}")
  else : print(f"Reject H0, there is correlation between {cat} and {target}")

In [ ]:
[checkanova(df,cat,'tip') for cat in "sex	smoker	day	time".split()]

p = 0.166456235034572 ,  accept H0, no correlation between sex and tip
p = 0.926593152224447 ,  accept H0, no correlation between smoker and tip
p = 0.17358855530405692 ,  accept H0, no correlation between day and tip
p = 0.05780153475171573 ,  accept H0, no correlation between time and tip


[None, None, None, None]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import pandas as pd

X = df[['total_bill',	'tip',	'sex',	'smoker',	'day',	'time',	'size']]

X_const = add_constant(X)

vif = pd.DataFrame()
vif['Variable'] = X_const.columns
vif['VIF'] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
vif

,Variable,VIF
0,const,16.540998
1,total_bill,2.369266
2,tip,1.882443
3,sex,1.061146
4,smoker,1.180482
5,day,1.932023
6,time,1.887897
7,size,1.684881


#1. Random Forest Regressor

In [ ]:
x = df.drop('tip',axis=1).values
y = df[['tip']].values
x.shape,y.shape

((244, 6), (244, 1))

In [ ]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100)

xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((195, 6), (49, 6), (195, 1), (49, 1))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfmodel = RandomForestRegressor(n_estimators=99, max_depth=3)
rfmodel.fit(xtrain,ytrain)

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=3, n_estimators=99)

In [ ]:
from sklearn.metrics import  r2_score
yhatrf = rfmodel.predict(xtest)

r2_score(ytest, yhatrf)

0.5437241262202127

In [ ]:
x = df.iloc[:,[0,6]].values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100)

rfmodel2 = RandomForestRegressor(n_estimators=99, max_depth=3)
rfmodel2.fit(xtrain,ytrain)
r2_score(ytest, rfmodel2.predict(xtest))

0.5170147121561296

#2 AdaBoost Regressor

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

x = df.drop('tip',axis=1).values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

adamodel = AdaBoostRegressor(n_estimators=11, learning_rate=0.5, random_state=42)
adamodel.fit(xtrain,ytrain)
r2_score(ytest, adamodel.predict(xtest))

0.5355107750953505

In [ ]:
x = df.iloc[:,[0,6]].values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

adamodel = AdaBoostRegressor(n_estimators=11, learning_rate=1, random_state=42)
adamodel.fit(xtrain,ytrain)
r2_score(ytest, adamodel.predict(xtest))

0.4551284157572627

#3 Gradient Boost Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

x = df.drop('tip',axis=1).values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

gbmodel = GradientBoostingRegressor(n_estimators=5, learning_rate=0.5, random_state=90)
gbmodel.fit(xtrain,ytrain)
r2_score(ytest, gbmodel.predict(xtest))

0.45741756260694966

In [ ]:
x = df.iloc[:,[0,6]].values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

gbmodel = GradientBoostingRegressor(n_estimators=5, learning_rate=0.5, random_state=100)
gbmodel.fit(xtrain,ytrain)
r2_score(ytest, gbmodel.predict(xtest))

0.32735863220399897

#4 XGBoost Regressor


In [ ]:
from xgboost import XGBRegressor

x = df.drop('tip',axis=1).values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

xgbmodel = XGBRegressor(n_estimators=100, learning_rate=0.01, random_state=42)
xgbmodel.fit(xtrain,ytrain)
r2_score(ytest, xgbmodel.predict(xtest))

0.40467511917478094

In [ ]:
x = df.iloc[:,[0,6]].values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

xgbmodel = XGBRegressor(n_estimators=100, learning_rate=0.01, random_state=42)
xgbmodel.fit(xtrain,ytrain)
r2_score(ytest, xgbmodel.predict(xtest))


0.22221520020495877

#5 LightGBM Regressor

In [ ]:
from lightgbm import LGBMRegressor

x = df.drop('tip',axis=1).values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

lgbmodel = LGBMRegressor(n_estimators=500, learning_rate=0.01, random_state=42);
lgbmodel.fit(xtrain,ytrain)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000188 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 82
[LightGBM] [Info] Number of data points in the train set: 195, number of used features: 6
[LightGBM] [Info] Start training from score 2.908513
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

LGBMRegressor(learning_rate=0.01, n_estimators=500, random_state=42)

In [ ]:
r2_score(ytest, lgbmodel.predict(xtest))

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


0.4247276545126012

In [ ]:
x = df.iloc[:,[0,6]].values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

lgbmodel = LGBMRegressor(n_estimators=500, learning_rate=0.01, random_state=42);
lgbmodel.fit(xtrain,ytrain)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000188 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 72
[LightGBM] [Info] Number of data points in the train set: 195, number of used features: 2
[LightGBM] [Info] Start training from score 2.908513
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

LGBMRegressor(learning_rate=0.01, n_estimators=500, random_state=42)

In [ ]:
r2_score(ytest, lgbmodel.predict(xtest))


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


0.44609683330866634

#6 CatBoost Regressor

In [ ]:

from catboost import CatBoostRegressor

x = df.drop('tip',axis=1).values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

catmodel = CatBoostRegressor(n_estimators=7, learning_rate=0.5, random_state=42);
catmodel.fit(xtrain,ytrain)

0:	learn: 1.0981636	total: 584us	remaining: 3.51ms
1:	learn: 1.0064211	total: 1.64ms	remaining: 4.09ms
2:	learn: 0.9180802	total: 2.18ms	remaining: 2.91ms
3:	learn: 0.8792193	total: 2.67ms	remaining: 2ms
4:	learn: 0.8579676	total: 2.87ms	remaining: 1.15ms
5:	learn: 0.8362874	total: 3.14ms	remaining: 522us
6:	learn: 0.7912242	total: 3.6ms	remaining: 0us


In [ ]:
r2_score(ytest, catmodel.predict(xtest))

0.42199823542047177

In [ ]:
x = df.iloc[:,[0,6]].values
y = df['tip'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

catmodel = CatBoostRegressor(n_estimators=7, learning_rate=0.5, random_state=42);
catmodel.fit(xtrain,ytrain)

0:	learn: 1.1167673	total: 392us	remaining: 2.35ms
1:	learn: 1.0181242	total: 999us	remaining: 2.5ms
2:	learn: 0.9959275	total: 1.18ms	remaining: 1.57ms
3:	learn: 0.9532075	total: 2.84ms	remaining: 2.13ms
4:	learn: 0.9237531	total: 3.31ms	remaining: 1.32ms
5:	learn: 0.9085053	total: 3.63ms	remaining: 604us
6:	learn: 0.8923910	total: 4.1ms	remaining: 0us


In [ ]:
r2_score(ytest, catmodel.predict(xtest))

0.4717955086974701

In [ ]:
#1 Random Forest Classifier

In [ ]:
df = sns.load_dataset('tips')

df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder


label = LabelEncoder()
for col in ['sex',"smoker", "day", "time"]:
    df[col] = label.fit_transform(df[col])

x = df.drop('sex',axis=1).values
y = df['sex'].values

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=100);

rfmodel = RandomForestClassifier(n_estimators=7, max_depth=7)
print(rfmodel.fit(xtrain,ytrain).score(xtest,ytest));

0.6530612244897959


In [ ]:
yhatrf = rfmodel.predict(xtest)
print(classification_report(ytest, yhatrf))

              precision    recall  f1-score   support

           0       0.30      0.23      0.26        13
           1       0.74      0.81      0.77        36

    accuracy                           0.65        49
   macro avg       0.52      0.52      0.52        49
weighted avg       0.63      0.65      0.64        49



#2 Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

nbmodel = GaussianNB()
nbmodel.fit(xtrain, ytrain)
nbyhat = nbmodel.predict(xtest)

print(classification_report(ytest, nbyhat))

              precision    recall  f1-score   support

           0       0.56      0.38      0.45        13
           1       0.80      0.89      0.84        36

    accuracy                           0.76        49
   macro avg       0.68      0.64      0.65        49
weighted avg       0.74      0.76      0.74        49



#3 Quadratic Discriminant Analysis

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qdamodel = QuadraticDiscriminantAnalysis()
qdamodel.fit(xtrain,ytrain).score(xtest,ytest)
yhatqda = qdamodel.predict(xtest)

print(classification_report(ytest, yhatqda))


              precision    recall  f1-score   support

           0       0.44      0.31      0.36        13
           1       0.78      0.86      0.82        36

    accuracy                           0.71        49
   macro avg       0.61      0.58      0.59        49
weighted avg       0.69      0.71      0.70        49

